In [1]:
import os
import time
import pickle
import pandas as pd
import numpy as np
import datetime

from gensim.utils import simple_preprocess, tokenize
from gensim.summarization.textcleaner import split_sentences
from gensim.parsing import preprocessing
from gensim.models import Doc2Vec
from gensim.corpora import Dictionary

from sklearn.cluster import KMeans

from tqdm import tqdm

import matplotlib.pyplot as plt
from multiprocessing import cpu_count

%matplotlib inline

In [3]:
df = pd.read_json('data/dev/cluster_12_cats_index.json')

In [5]:
doc_vecs = pickle.load(open("data/prod/doc_vecs.pkl", "rb"))

In [6]:
dataset = pickle.load(open("data/prod/dataset_2_docs.pkl", "rb"))

In [7]:
def get_doc_vecs(doc_vecs, df, ids):
    df2 = df.reset_index()
    df2 = df2.set_index('id')

    indexes = df2.loc[ids, 'index'].tolist()
    # print(len(indexes))
    vecs = doc_vecs[indexes]
    
    return vecs


def kmean_cluster(doc_vecs, df, samples, n_clusters=20):
    models = []
    
    for i, sample in tqdm(enumerate(samples)):
        #sample.reset_index(inplace=True, drop=True)
        vecs = get_doc_vecs(doc_vecs, df, sample['id'])
        
        model = KMeans(n_clusters=n_clusters, random_state=2).fit(vecs)
        models.append(model)
        
    return models

In [ ]:
models = kmean_cluster(doc_vecs, df, dataset, n_clusters=20)

In [ ]:
pickle.dump(models, open('data/prod/kmean_1k_models.pkl', 'wb'))